# Assignment 4 (solutions): Attention

### Due Date: Nov 22 (both sections)

### Grade (100 pts, 10%)

#### Your Name:Qinren Zhou

#### Your EID: qz142

*Note: This assignment covers material from the recording, notes, demo, and suggested readings from Lectures 5,8,9*

---

### Problem introduction

In this assignment we will explore the concept of attention using Word2Vec features. Recall that in lecture-05 we used the Word2Vec algorithm to compute feature representations of english words. For every center-word - context-word pair, ${\textbf{x}_w, \textbf{x}_c}$, in the training set, Word2Vec tries to maximize the inner product between their feature embeddings, $\textbf{u}_w$ and $\textbf{v}_c$, by first computing a vector of logits as $\textbf{z} = \textbf{u}_w \cdot \textbf{V}^T \in \mathbb{R}^N$, and then maximizing $p(\textbf{x}_c | \textbf{x}_w) = \sigma_{softmax}(\textbf{z})$ to learn $\textbf{U}$ and $\textbf{V}$ via MLE. 

Let's assume we are given a sequence of words:

$$X = \{ \textbf{x}^{(1)}, \textbf{x}^{(2)}, \textbf{x}^{(3)}, \textbf{x}^{(4)}, \dots, \textbf{x}^{(T)} \}$$

a corresponding sequence of center-word feature representations:

$$U = \{ \textbf{u}^{(1)}, \textbf{u}^{(2)}, \textbf{u}^{(3)}, \textbf{u}^{(4)}, \dots, \textbf{u}^{(T)} \}$$

and a corresponding context-word feature representations:

$$V = \{ \textbf{v}^{(1)}, \textbf{v}^{(2)}, \textbf{v}^{(3)}, \textbf{v}^{(4)}, \dots, \textbf{v}^{(T)} \}$$

Then, using the Word2Vec variable naming convention above, recall that in simple self attention (Lecture-09) we compute a set of *context vectors*:

$$C = \{ \textbf{c}^{(1)}, \textbf{c}^{(2)}, \textbf{c}^{(3)}, \textbf{c}^{(4)}, \dots, \textbf{c}^{(T)} \}$$ 

each of which is computed according to:

$$
\textbf{c}^{(t)} = \sum_{t'=1}^{T} \alpha_{t,t'} \; \textbf{u}^{(t')} \quad where \quad \alpha_{t,t'} = \frac{ e^{ \textbf{u}^{(t)} \cdot \textbf{v}^{(t')} } }{ \sum_{t''} e^{ \textbf{u}^{(t)} \cdot \textbf{v}^{(t'')} } } \\
$$

The attention weights make up the following (non-symmetric!) matrix:

$$
\textbf{A} = 
\left[ \begin{array}{ccc}
\alpha_{1,1} & \dots & \alpha_{1,T} \\
\vdots & \ddots & \vdots \\
\alpha_{T,1} & \dots & \alpha_{T,T}
\end{array}\right] \; \in [0,1]^{T \times T} \quad where \quad \sum_{t'} \alpha_{t,t'} = 1
$$

### Test sentence

In [36]:
sentence = "Thomas Jefferson was an American statesman and Founding Father who served as the third president of the United States"

### Load Word2Vec vocab, embedding matrices

In the `assignments/a4/` folder you will find three .tsv files: a vocabulary (`metadata.tsv`), center-word Word2Vec embeddings (`vectors_center.tsv`), and context-word Word2Vec embeddings (`vectors_context.tsv`). These are taking from the Lecture-05 Word2Vec demo. The code below loads the center and context word embeddings for the above test sentence into two numpy arrays: $U$, and $V$. 

In [37]:
import numpy as np

def load_matrix(fpath):
    matrix = []
    with open(fpath, 'r') as fd:
        tsv = fd.read()
    for line in tsv.split('\n'):
        row = []
        for value in line.split('\t'):
            row.append(float(value))
        matrix.append(row)
    return np.array(matrix)

def load_vocab(fpath) -> dict: # metadata.tsv
    with open(fpath, "r") as fd:
        tsv = fd.read()
    vocab = {}
    for line in tsv.split('\n'):
        vocab[line.strip()] = len(vocab)
    return vocab

vocab = load_vocab("metadata.tsv")
U_lookup = load_matrix("vectors_center.tsv").T
V_lookup = load_matrix("vectors_context.tsv").T

N = len(vocab)
D = len(U_lookup)

print(f"vocab size: {N}")
print(f"embed dim: {D}")

print(f"center embedding lookup matrix shape: {U_lookup.shape}")
print(f"context embedding lookup matrix shape: {V_lookup.shape}")

vocab size: 2321
embed dim: 15
center embedding lookup matrix shape: (15, 2321)
context embedding lookup matrix shape: (15, 2321)


In [38]:
vocab.get('president', N)

1622

In [39]:
sentence = "Thomas Jefferson was an American statesman and Founding Father who served as the third president of the United States"
word_list = sentence.lower().split(" ")
X = [vocab.get(word, N) for word in word_list]

print(f"Word indices: {X}")
print(len(X))

Word indices: [2092, 1075, 2321, 2321, 79, 2321, 2321, 795, 2321, 2321, 2321, 2321, 2321, 2321, 1622, 2321, 2321, 2158, 1987]
19


In [40]:
# Assign OOV words low attention weights
u_oov = 0.1 * np.mean(U_lookup, axis=1, keepdims=True)
v_oov = 0.1 * np.mean(V_lookup, axis=1, keepdims=True) 
# Add the OOV embeddings to the lookup matrices
U_lookup = np.concatenate((U_lookup, u_oov), axis=1)
V_lookup = np.concatenate((V_lookup, v_oov), axis=1)
U_lookup.shape, V_lookup.shape

print(f"center word embedding lookup matrix w/oov embed added shape: {U_lookup.shape}")
print(f"context word embedding lookup matrix w/oov embed added shape: {U_lookup.shape}")

center word embedding lookup matrix w/oov embed added shape: (15, 2322)
context word embedding lookup matrix w/oov embed added shape: (15, 2322)


In [41]:
# Center-word embedding sequence of X
U = U_lookup[:, X]
# Context-word embedding sequence of X
V = V_lookup[:, X]

print(f"sequence of center-word embeddings shape: {U.shape}")
print(f"sequence of context-word embeddings shape: {U.shape}")

sequence of center-word embeddings shape: (15, 19)
sequence of context-word embeddings shape: (15, 19)


# Questions

### (40 pts) Q1: Attention weights

1. Compute the attention weight matrix $\textbf{A}$ for this test sentence using the definitions provided in the intro (and in Lecture-09 as needed).

*Hint: You can use the batched `softmax()` function provided below to compute A.*

In [42]:
def softmax(Z) -> np.ndarray:
    """
    Computes a softmax over each row of Z
    """
    Z_exp = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    partition = np.sum(Z_exp, axis=1, keepdims=True)
    return Z_exp / partition

In [43]:
# len of sentence: 19
# dim len : 15
A = []
for t in range(len(X)):
    ut  = U[:,t]
    lis = []
    for tt in range(len(X)):
        vtt = V[:,tt]
        lis.append(np.exp(np.dot(ut,vtt)))
    sum_tmp = sum(lis)
    lis = [round(i/sum_tmp,6) for i in lis]
    A.append(lis)
    
print(len(A),len(A[0]))
#print(A)

19 19


### (40 pts) Q2: Attention block

1. Now compute the attention block $C$ using the attention weights that you've computed

2. Compute the L2 norm of each of the resultant context vectors. How do the magnitudes compare? Do the same for the center-word embeddings in the preceeding layer. Is there a logical explanation for this?

*Hint: You need to compute $\textbf{c}^{(t)} \; \forall t \in \{1, \dots, T\}$.*

In [44]:
def list_add(lisA,lisB):
    res = []
    for i in range(len(lisA)):
        res.append(lisA[i] + lisB[i])
    return res


C = []
for t in range(len(U[0])):      # loop U by col
    ct = [0]*len(U[0])
    for tt in range(len(U[0])): # loop each element in A
        a = A[t][tt]            # a(t,t')
        tmp = [i * a for i in U[:,tt]]  # a(t,t') * U{t}
        ct = list_add(tmp, ct)          # sum, get C(t)
    C.append(ct)
C = np.array(C)
C = C.T
print(len(C),len(C[0]))
#print(C)

15 19


In [45]:
# L2 for C 
C = C.T
for i in range(len(C)):
    vec = np.array(C[i])
    vec = np.linalg.norm(vec)
    print(i,vec)
# each line represent the L2 normal for attention weights C(t)

0 2.155972097134811
1 2.2276660321735493
2 1.1225692169726431
3 1.1225692169726431
4 2.27455988446378
5 1.1225692169726431
6 1.1225692169726431
7 2.3455183239303006
8 1.1225692169726431
9 1.1225692169726431
10 1.1225692169726431
11 1.1225692169726431
12 1.1225692169726431
13 1.1225692169726431
14 2.18157218083374
15 1.1225692169726431
16 1.1225692169726431
17 2.4893443419038395
18 2.482837573127786


In [46]:
U = U.T 
for i in range(len(U)):
    vec = np.array(U[i])
    vec = np.linalg.norm(vec)
    print(i,vec)
# each line represent the L2 normal for center-word embeddings U(t)

0 2.7284433320768997
1 2.971433701326006
2 0.18668971616635238
3 0.18668971616635238
4 2.2807288702742743
5 0.18668971616635238
6 0.18668971616635238
7 2.6194201449887724
8 0.18668971616635238
9 0.18668971616635238
10 0.18668971616635238
11 0.18668971616635238
12 0.18668971616635238
13 0.18668971616635238
14 2.9704852555359977
15 0.18668971616635238
16 0.18668971616635238
17 2.535290847011032
18 2.532197722371995


After L2 normal, the result of each attention weights become more stable, compared to visible fluctuations in center-word embeddings

### (20 pts) Q3: Visualize the attention weights

Now that we've computed a simple self attention transformation of our Word2Vec sequence representation, it's time to visually examine what this attention mechanism affords us. First recognize that all values in $\textbf{A}$ are in the range $[0,1]$, and that each row is normalized. Therefore, we really just need to generate a heatmap of this matrix, keeping in mind that the matrix indices correspond directly to the positions in the sequence.

1. Generate the heatmap in the cell provided below

2. Briefly describe/explain what you see in the heatmap.

3. Why is our attention matrix not symmetric?

*Hint: You can generate a heatmap of the attention weights by calling `plot_attention_weights(A)`.*

In [47]:
import plotly.express as px

def plot_attention_weights(att_matrix):
    fig = px.imshow(A, x=word_list, y=word_list, width=20, height=20)
    fig.update_layout(width=800, height=800)
    print(sentence)
    fig.show()

In [48]:
# Your code goes here

# make sure each element in A is in the range  [0,1] and each row is normalized
lis  = []
for i in A:
    line_sum = 0
    for j in i:
        line_sum += j
    lis.append(line_sum)  # it should be near 1
print(lis)
plot_attention_weights(A)

[1.0000049999999998, 0.9999969999999998, 0.999997, 0.999997, 1.0000040000000001, 0.999997, 0.999997, 0.999999, 0.999997, 0.999997, 0.999997, 0.999997, 0.999997, 0.999997, 0.999997, 0.999997, 0.999997, 0.9999990000000001, 1.000001]


TypeError: imshow() got an unexpected keyword argument 'x'

The words [Thomas,Jefferson,founding,president], pay more attention to the word 'United States' and 'American' .
Intuitively understand, because Thomas Jefferson often appears with the  'United States' or 'American' in the text,
And this verb 'founding' generally modify the nouns of a country or organization
as well as the word 'presidential' is bound to the concept of a country or organization.

Besides, proper nouns and key verbs tend to get more attention, while other part-of-speech words are less important in sentences.

As for synonyms, words between common phrase groups also share more attention. 
In Word2vec, words with similar context will be closer together in the embedded representation.

In terms of why attention matrix not symmetric. The intuitive understanding is that in a sentence, 
the attention of A word to B word is not necessarily equal to the attention of B word to A word. 
For example, for words 'founding', Ignore other words and assuming the word 'American' and 'United States' share same attention, 0.5 by 0.5 for example.
However, when considering the word 'American', the distance between 'American' and 'United States' will be much closer compared to 'American' and 'United States'
Thus, the word 'American' may pay more attention to 'United States', 0.3 by 0.7 for example.

Another way to understand mathematics is to change the matrix to calculate row by row, and the sum of each row is 1. When calculating the first row (that is, the attention of the first word), it has nothing to do with the attention of other words. , Calculated separately and independently